In [ ]:
import streamlit as st
import pandas as pd
import joblib
import pickle
import datetime
import sklearn
import os.path
from sklearn.metrics import accuracy_score
import pycaret
from pycaret.classification import load_model, setup, predict_model
import base64

######################################################################################################################################################################

# Título y subtitulo
st.title("Modelos Banco de Bogotá")
st.subheader("Sube aquí tus archivos")

####################################################################### CARGA DE ARCHIVOS ###############################################################################################

# Subir archivo desde PC
uploaded_file = st.file_uploader("Sube un archivo CSV")

# Carga de archivo
df1 = None

if uploaded_file is not None:
    try:
        df1 = pd.read_csv(uploaded_file, sep = ";")
        st.success("¡Archivo cargado exitosamente!")
        st.write("Primeros 5 registros del Archivo:")
        st.write(df1.head(5))
    except:
        st.error("Ocurrió un error al cargar el archivo")


######################################################################################################################################################################

st.title("Hora de contacto")

# Función de transformación de datos
@st.cache_data
def transformacion_datos(df):
    # Llamamos a la función preparacion_datos para transformar el df
    transformed_data = preparacion_datos(df)
    return transformed_data

# Función preparacion_datos
def preparacion_datos(df1):
  # convertir la columna 'Fecha de Cumpleaños' a formato datetime
  df1['Fecha_cumpleanios'] = pd.to_datetime(df1['Fecha_cumpleanios'], format='%d/%m/%Y')
  # fecha actual
  hoy = datetime.date.today()
  # calcular la edad para cada fila y agregar una nueva columna llamada 'Edad'
  df1['EDAD'] = df1['Fecha_cumpleanios'].apply(lambda x: hoy.year - x.year - ((hoy.month, hoy.day) < (x.month, x.day)))
  df1['EDAD'] = df1['EDAD'].fillna(0)
  df1['EDAD'] = df1['EDAD'].astype(int)
  # Solucionamos valores vacios
  df1['Perfil_facturacion_Top1_Top2'] = df1['Perfil_facturacion_Top1_Top2'].fillna('NOTIENE/NO/NO')
  # crear una nueva columna con los valores separados por '/'
  split_data = df1['Perfil_facturacion_Top1_Top2'].str.split('/', expand=True)
  # agregar las nuevas columnas una a una para evitar el error
  df1['PERFIL_FACTURACION'] = split_data[0]
  df1['TOP1'] = split_data[1]
  df1['TOP2'] = split_data[2]
  # eliminar la columna original
  df1.drop(columns=['Perfil_facturacion_Top1_Top2'], inplace=True)
  # Convierte la columna de fechas a un objeto datetime para contar días desde ultima compra de cartera
  df1['Fecha_ultima_compra_cartera'] = pd.to_datetime(df1['Fecha_ultima_compra_cartera'], format='%d/%m/%Y')
  hoy = datetime.date.today()
  df1['DIAS_ULTIMA_COMPRA'] = df1.fillna({'Fecha_ultima_compra_cartera': 0})['Fecha_ultima_compra_cartera'].apply(lambda x: (hoy - x.date()).days if x != 0 else 0)
  df1['Oferta_compra_cartera'] = df1['Oferta_compra_cartera'].fillna(0)
  df1['TASA_ACEPTADA'] = df1['TASA_ACEPTADA'].str.replace(',', '.')
  #Preparación de input para el modelo
  Train = df1.loc[:,['DOCUMENTO','GENERO','PROFESION','CIUDAD','ENDEUDAMIENTO','1_PRODUCTO_ACTUAL','TASA_ACEPTADA','1_FRANQUICIA_TC','Saldo_total',
                'Ciclo', 'Marca_retencion','Marca_FGA', 'Oferta_compra_cartera','EDAD', 'PERFIL_FACTURACION', 'TOP1', 'TOP2', 'DIAS_ULTIMA_COMPRA']]
  #Validamos tipos de dato
  Train = Train.astype({'DOCUMENTO'                  :'int64',
                        'GENERO'                     :'category',
                        'PROFESION'                  :'category',
                        'CIUDAD'                     :'category',
                        'ENDEUDAMIENTO'              :'category',
                        '1_PRODUCTO_ACTUAL'          :'category',
                        'TASA_ACEPTADA'              :'float64',
                        '1_FRANQUICIA_TC'            :'category',
                        'Saldo_total'                :'int64',
                        'Ciclo'                      :'int64',
                        'Marca_retencion'            :'category',
                        'Marca_FGA'                  :'category',
                        'Oferta_compra_cartera'      :'float64',
                        'EDAD'                       :'int64',
                        'PERFIL_FACTURACION'         :'category',
                        'TOP1'                       :'category',
                        'TOP2'                       :'category',
                        'DIAS_ULTIMA_COMPRA'         :'int64'})
  #Valores vacios
  #Valores que vamos a reemplazar por 0
  df1.fillna({'Fecha_ultima_compra_cartera':0,'E_TC_COMPRA_CARTERA':0,'FECHA_DE_NACIMIENTO':0,'Fecha_ultima_compra_cartera':0}, inplace = True)
  #Valores que vamos a reemplazar por la moda
  moda_profesion = df1['PROFESION'].mode()[0]
  moda_semaforo = df1['COLOR_SEMAFORO'].mode()[0]
  moda_top1 = df1['TOP1'].mode()[0]
  moda_top2 = df1['TOP2'].mode()[0]
  #Hacemos el reemplazo
  df1['PROFESION'] = df1['PROFESION'].fillna(moda_profesion)
  df1['COLOR_SEMAFORO'] = df1['COLOR_SEMAFORO'].fillna(moda_semaforo)
  df1['TOP1'] = df1['TOP1'].fillna(moda_top1)
  df1['TOP2'] = df1['TOP1'].fillna(moda_top2)
  df1['Saldo_total'] = df1['Saldo_total'].apply(lambda x: round(x))
  #Generamos nuestro conjunto final
  Train = df1.loc[:,['DOCUMENTO', 'GENERO','PROFESION', 'CIUDAD', 'ENDEUDAMIENTO', '1_PRODUCTO_ACTUAL',
                          'TASA_ACEPTADA', '1_FRANQUICIA_TC','PLAZO_ACEPTADO', 'TIPO_DE_DEUDA_A_COMPRAR','E_CONFIRMA_CLIENTE','Saldo_total',
                          'Ciclo','Marca_retencion', 'Oferta_compra_cartera', 'EDAD', 'PERFIL_FACTURACION', 'TOP1','TOP2', 'DIAS_ULTIMA_COMPRA']]
  return Train

# Cargar el DataFrame previamente cargado
df = df1.copy()
# Cargar el DataFrame previamente cargado
df2 = df1.copy()

# Crear un botón que ejecute la función de transformación
if st.button('Transformacion 1'):
    # Llamar a la función de transformación y guardar el resultado en un nuevo DataFrame
    new_df = transformacion_datos(df)
    # Mostrar el nuevo DataFrame resultante
    st.write(new_df.head(5))

model = load_model("C:/Streamlit/modelo_hora")

cc = transformacion_datos(df)
doc = cc.loc[:,['DOCUMENTO']]
new_df = cc.drop(['DOCUMENTO','TIPO_DE_DEUDA_A_COMPRAR','E_CONFIRMA_CLIENTE'], axis=1)


# Función para realizar las predicciones
def hacer_predicciones(model, data):
    predictions = predict_model(model, data)
    return predictions

# Interfaz de Streamlit
def main():
    # Botón para realizar las predicciones y mostrar los resultados
    if st.button("Predicciones Hora"):
        results = hacer_predicciones(model, new_df)
        results_hora = results.copy()
        st.write(results)
        df_concatenado = pd.concat([doc,results], axis=1)
        csv = df_concatenado.to_csv(index=False)
        b64 = base64.b64encode(csv.encode()).decode()
        href = f'<a href="data:file/csv;base64,{b64}" download="resultados.csv">Descargar CSV</a>'
        st.markdown(href, unsafe_allow_html=True)

if __name__ == "__main__":
  main()


######################################################################################################################################################################

st.title("Probabilidad CC")

# Función de transformación de datos 2
@st.cache_data
def transformacion_datos2(df2):
    # Llamamos a la función preparacion_datos para transformar el df
    transformed_data2 = preparacion_datos2(df2)
    return transformed_data2

# Función preparacion_datos 2
def preparacion_datos2(df2):
  # convertir la columna 'Fecha de Cumpleaños' a formato datetime
  df2['Fecha_cumpleanios'] = pd.to_datetime(df2['Fecha_cumpleanios'], format='%d/%m/%Y')
  # fecha actual
  hoy = datetime.date.today()
  # calcular la edad para cada fila y agregar una nueva columna llamada 'Edad'
  df2['EDAD'] = df2['Fecha_cumpleanios'].apply(lambda x: hoy.year - x.year - ((hoy.month, hoy.day) < (x.month, x.day)))
  df2['EDAD'] = df2['EDAD'].fillna(0)
  df2['EDAD'] = df2['EDAD'].astype(int)
  # Solucionamos valores vacios
  df2['Perfil_facturacion_Top1_Top2'] = df2['Perfil_facturacion_Top1_Top2'].fillna('NOTIENE/NO/NO')
  # crear una nueva columna con los valores separados por '/'
  split_data = df2['Perfil_facturacion_Top1_Top2'].str.split('/', expand=True)
  # agregar las nuevas columnas una a una para evitar el error
  df2['PERFIL_FACTURACION'] = split_data[0]
  df2['TOP1'] = split_data[1]
  df2['TOP2'] = split_data[2]
  # eliminar la columna original
  df1.drop(columns=['Perfil_facturacion_Top1_Top2'], inplace=True)
  # Convierte la columna de fechas a un objeto datetime
  df2['Fecha_ultima_compra_cartera'] = pd.to_datetime(df2['Fecha_ultima_compra_cartera'], format='%d/%m/%Y')
  hoy = datetime.date.today()
  df2['DIAS_ULTIMA_COMPRA'] = df2.fillna({'Fecha_ultima_compra_cartera': 0})['Fecha_ultima_compra_cartera'].apply(lambda x: (hoy - x.date()).days if x != 0 else 0)
  df2.head(5)
  # eliminar la columna original
  df2.drop(columns=['Fecha_ultima_compra_cartera'], inplace=True)
  # Convierte la columna de fechas a un objeto datetime
  df2['Ultima _fecha _pago'] = pd.to_datetime(df2['Ultima _fecha _pago'], format='%d/%m/%Y')
  hoy = datetime.date.today()
  df2['DIAS_ULTIMO_PAGO'] = df2.fillna({'Ultima _fecha _pago': 0})['Ultima _fecha _pago'].apply(lambda x: (hoy - x.date()).days if x != 0 else 0)
  # eliminar la columna original
  df2.drop(columns=['Ultima _fecha _pago'], inplace=True)
  df2.head(5)
  # Reemplazo de valores vacios
  df2['Oferta_compra_cartera'] = df2['Oferta_compra_cartera'].fillna(0)
  df2['Oferta_reto_meta_fact_monto'] = df2['Oferta_reto_meta_fact_monto'].fillna(0)
  df2['Oferta_reto_Meta_transaccion'] = df2['Oferta_reto_Meta_transaccion'].fillna(0)
  df2['DIAS_ULTIMO_PAGO'] = df2['DIAS_ULTIMO_PAGO'].fillna(0)
  # Redondear valores al número entero más cercano
  df2['Oferta_reto_meta_fact_monto_2'] = df2['Oferta_reto_meta_fact_monto'].round().astype(int)
  df2['Oferta_reto_Meta_transaccion_2'] = df2['Oferta_reto_Meta_transaccion'].round().astype(int)
  df2['Disponible_2'] = df2['Disponible'].round().astype(int)
  df2['Pago_minimo_2'] = df2['Pago_minimo'].round().astype(int)
  df2['Saldo_mora_2'] = df2['Saldo_mora'].round().astype(int)
  df2['TASA_ACEPTADA'] = df2['TASA_ACEPTADA'].str.replace(',', '.')
  # Preparacion conjunto de entrenamiento
  Train2 = df2.loc[:,['DOCUMENTO', 'GENERO','PROFESION', 'CIUDAD', 'ENDEUDAMIENTO', '1_PRODUCTO_ACTUAL',
                        'TASA_ACEPTADA', '1_FRANQUICIA_TC','PLAZO_ACEPTADO', 'TIPO_DE_DEUDA_A_COMPRAR','E_CONFIRMA_CLIENTE','Saldo_total',
                        'Ciclo','Marca_retencion', 'Oferta_compra_cartera', 'EDAD','Cupo','Disponible_2','Saldo_mora_2','Pago_minimo_2','Valor_ cuota_manejo',
                        'Oferta_reto_meta_fact_monto_2','Oferta_reto_Meta_transaccion_2','PERFIL_FACTURACION', 'TOP1','TOP2', 'DIAS_ULTIMA_COMPRA','DIAS_ULTIMO_PAGO']]
  Train2 = Train2.astype({'DOCUMENTO'                    :'int64',
                      'GENERO'                       :'category',
                      'PROFESION'                    :'category',
                      'CIUDAD'                       :'category',
                      'ENDEUDAMIENTO'                :'category',
                      '1_PRODUCTO_ACTUAL'            :'category',
                      'TASA_ACEPTADA'                :'float64',
                      '1_FRANQUICIA_TC'              :'category',
                      'Saldo_total'                  :'int64',
                      'Ciclo'                        :'int64',
                      'Marca_retencion'              :'category',
                      'Oferta_compra_cartera'        :'float64',
                      'EDAD'                         :'int64',
                      'Cupo'                         :'int64',
                      'Disponible_2'                   :'int64',
                      'Saldo_mora_2'                   :'int64',
                      'Pago_minimo_2'                  :'int64',
                      'Valor_ cuota_manejo'          :'int64',
                      'Oferta_reto_meta_fact_monto_2'  :'int64',
                      'Oferta_reto_Meta_transaccion_2' :'int64',
                      'PERFIL_FACTURACION'           :'category',
                      'TOP1'                         :'category',
                      'TOP2'                         :'category',
                      'DIAS_ULTIMA_COMPRA'           :'int64',
                      'DIAS_ULTIMO_PAGO'             :'int64'})
  #Valores que vamos a reemplazar por 0
  df1.fillna({'Fecha_ultima_compra_cartera':0,'E_TC_COMPRA_CARTERA':0,'FECHA_DE_NACIMIENTO':0,'Fecha_ultima_compra_cartera':0}, inplace = True)
  Train2 = df2.loc[:,['DOCUMENTO', 'GENERO','PROFESION', 'CIUDAD', 'ENDEUDAMIENTO', '1_PRODUCTO_ACTUAL',
                         'TASA_ACEPTADA', '1_FRANQUICIA_TC','PLAZO_ACEPTADO', 'TIPO_DE_DEUDA_A_COMPRAR','Saldo_total',
                         'Ciclo','Marca_retencion', 'Oferta_compra_cartera', 'EDAD','Cupo','Disponible_2','Saldo_mora_2','Pago_minimo_2','Valor_ cuota_manejo',
                         'Oferta_reto_meta_fact_monto_2','Oferta_reto_Meta_transaccion_2','PERFIL_FACTURACION', 'TOP1','TOP2', 'DIAS_ULTIMA_COMPRA','DIAS_ULTIMO_PAGO']]
  return Train2

# Cargar el DataFrame previamente cargado
#df2 = df1.copy()

# Crear un botón que ejecute la función de transformación
if st.button('Transformacion 2'):
    # Llamar a la función de transformación y guardar el resultado en un nuevo DataFrame
    new_df2 = transformacion_datos2(df2)
    # Mostrar el nuevo DataFrame resultante
    st.write(new_df2.head(5))

model1 = load_model("C:/Streamlit/modelo_probabilidad")

cc1 = transformacion_datos2(df2)
doc1 = cc1.loc[:,['DOCUMENTO']]
new_df2 = cc1.drop(['DOCUMENTO'], axis=1)

# Función para realizar las predicciones
def hacer_predicciones2(model1, data):
    predictions2 = predict_model(model1, data)
    return predictions2

# Interfaz de Streamlit
def main2():
    # Botón para realizar las predicciones y mostrar los resultados
    if st.button("Predicciones probabilidad"):
        results2 = hacer_predicciones2(model1, new_df2)
        results_prob = results2.copy()
        st.write(results2)
        df_concatenado2 = pd.concat([doc1,results2], axis=1)
        csv = df_concatenado2.to_csv(index=False)
        b64 = base64.b64encode(csv.encode()).decode()
        href = f'<a href="data:file/csv;base64,{b64}" download="resultados.csv">Descargar CSV</a>'
        st.markdown(href, unsafe_allow_html=True)

if __name__ == "__main__":
  main2()

##########################################################################################################################################################################################


